# **Import Statements:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = "/content/drive/My Drive/CS196Project/Group6"
%cd $BASE_PATH

# BASE_PATH = "/Users/neilkaushikkar/Desktop/Code/UIUC/CS196/Project/Group6"
# %cd $BASE_PATH
# !pwd

!pip install -r ./finetuning/requirements.txt
# !pip install pytorch_lightning
# !pip install gitpython
# !pip install rouge-score
# !pip install sacrebleu
!pip install datasets
!pip install transformers
# !pip install torch
# !pip install transformers
# !pip install lightning_base

from datasets import load_dataset
import torch

In [3]:
torch.cuda.empty_cache()
torch.cuda.is_available()

True

# **Load Datasets:**

 TODO: format newsroom
 
 TODO: format custom dataset

In [4]:
newsroom = load_dataset('newsroom', data_dir='/content/drive/My Drive/CS196Project/newsroom')

Using custom data configuration default


Dataset newsroom downloaded and prepared to /root/.cache/huggingface/datasets/newsroom/default/1.0.0/4b405ccd64e15f685065870ea563a1e6a034d1bd269a5427f40146d81549095e. Subsequent calls will reuse this data.


Create Custom Dataset:

# **Testing Transformers**

In [ ]:
%%bash
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/finetune.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/callbacks.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/utils.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/sentence_splitter.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/lightning_base.py
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/requirements.txt


mkdir example_data && cd example_data

wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/test.source
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/test.target
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/train.source
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/train.target
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/val.source
wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/test_data/wmt_en_ro/val.target

cd ..

In [9]:
%%bash
# BASE_PATH="/Users/neilkaushikkar/Desktop/Code/UIUC/CS196/Project/Group6"
BASE_PATH="/content/drive/My Drive/CS196Project/Group6"

cd "/content/drive/My Drive/CS196Project/Group6"
pwd
cd finetuning
pwd

    # --do_predict \
    # --eval_batch_size=1 \

    # --freeze_embeds \
    # --fp16 \


export OUTPUT_DIR="example_output"
export DATA_DIR="example_data"

python finetune.py \
    --learning_rate=3e-5 \
    --gpus 1 \
    --do_train \
    --n_val 1000 \
    --val_check_interval 0.1 \
    --data_dir=$DATA_DIR \
    --train_batch_size=1 \
    --output_dir=$OUTPUT_DIR \
    --overwrite_output_dir \
    --freeze_encoder \
    --num_train_epochs 6 \
    --max_target_length=60 \
    --val_max_target_length=60 \
    --test_max_target_length=100 \
    --model_name_or_path sshleifer/distilbart-xsum-12-6
    "$@"

/content/drive/My Drive/CS196Project/Group6
/content/drive/My Drive/CS196Project/Group6/finetuning
Epoch 0:   7%|▋         | 2/30 [00:00<00:09,  3.05it/s, loss=7.874, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  17%|█▋        | 5/30 [01:03<05:17, 12.69s/it, loss=7.471, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  27%|██▋       | 8/30 [01:11<03:16,  8.91s/it, loss=7.132, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  37%|███▋      | 11/30 [01:18<02:15,  7.13s/it, loss=7.019, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  47%|████▋     | 14/30 [01:25<01:37,  6.12s/it, loss=6.840, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  57%|█████▋    | 17/30 [01:33<01:11,  5.50s/it, loss=6.650, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  67%|██████▋   | 20/30 [01:40<00:50,  5.05s/it, loss=6.503, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  77%|███████▋  | 23/30 [01:48<00:33,  4.74s/it, loss=6.275, v_num=0]
Validating: 0it [00:00, ?it/s]
Epoch 0:  87%|████████▋ | 26/30 

2020-11-04 08:24:28.863657: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /content/drive/My Drive/CS196Project/Group6/finetuning/example_output exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was dep